In [6]:

import argparse
# import http.client
# import httplib2
import os
import random
import time
import datetime
from datetime import datetime, timedelta
# import google.oauth2.credentials
# import google_auth_oauthlib.flow
# from googleapiclient.discovery import build
# from googleapiclient.errors import HttpError
# from google_auth_oauthlib.flow import Flow, InstalledAppFlow
# from googleapiclient.http import MediaFileUpload, MediaIoBaseDownload
# from google.auth.transport.requests import Request
# from DashboardFunctions import *
import os
from dotenv import load_dotenv
from sqlalchemy import *
import datetime
from Google import *
import re
from IPython.display import Image
load_dotenv()
from StreamMaster import *
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from twitchAPI.twitch import Twitch
from twitchAPI.types import VideoType
from copy import copy, deepcopy
from jupyter_dash import JupyterDash
import dash
from dash import dcc, Dash
from dash import html
from matplotlib.colors import *
import random
import regex as re
from pydub import AudioSegment
from _utils import *
import glob
from collections import defaultdict
from string import punctuation

In [10]:
test = defaultdict(int)

In [8]:
test_string = "Hi, my name is Avik!"

In [15]:
test_string.translate(None, punctuation)

TypeError: str.translate() takes exactly one argument (2 given)

In [17]:
test_string.translate(str.maketrans('','', punctuation))

'Hi my name is Avik'

In [11]:
test['hi'] += 1

In [3]:
timestamps_to_string(
generate_timestamps(mysql_engine, " Campfire Cookoff"))

'00:00:00 - Intro\n00:00:06 - 1-4\n00:04:24 - Kevin 1\n00:09:16 - 2-1\n00:13:47 - 2-2\n'

In [5]:
with Session(mysql_engine) as session:
    print(session.query(Game_Meta.game_id, Game_Meta.game_name).where(Game_Meta.game_name.ilike('%Overcooked%')).all())

[(518847, 'Overcooked! All You Can Eat')]


In [21]:
title = " Campfire Cookoff"
mode='QUEST'
with Session(mysql_engine) as session:
    print(session.query(TwitchVideos.duration, TwitchVideos.video_name, strtString).where(TwitchVideos.video_type == TwitchVideoStyle.HIGHLIGHT).order_by(endString, highlightModeIx).where(highlightMode == mode).where(endString == title).all())

[(255.0, '1-4 | Quest 1 | Campfire Cookoff.mp4', '1-4 '), (289.0, 'Kevin 1 | Quest 2 | Campfire Cookoff.mp4', 'Kevin 1 '), (268.0, '2-1 | Quest 3 | Campfire Cookoff.mp4', '2-1 '), (253.0, '2-2 | Quest 4 | Campfire Cookoff.mp4', '2-2 ')]


In [16]:
with Session(mysql_engine) as session:
    for clip_id, name in session.query(TwitchVideos.id, endString).where(endString == ' Crash Landing').all():
        session.execute(update(TwitchVideos).where(TwitchVideos.id == clip_id).values({'published': None}))
        session.commit()

In [6]:
from sqlalchemy import engine

In [2]:

def clear_tempfiles(root: str = 'tmp_folder/*_ts.mp4'):
    """Remove temp files"""
    [os.remove(path) for path in glob.glob(root)]

In [35]:
clear_tempfiles()

In [ ]:
with Session(mysql_engine) as session:
    # session.execute(delete(Clip_Tracker ).where(Clip_Tracker.title.ilike('%RoundWon rating%')), execution_options={"synchronize_session": 'fetch'})
    # print(session.query(Clip_Tracker).where(Clip_Tracker.title.ilike('%RoundWon rating%')).all())
    # print(session.query(Game_Meta.game_name).where(Game_Meta.game_name.ilike("%Call of %")).all())
    
    # session.commit()

In [2]:
# Key Connection Variables
sql_engine = create_engine(f"mysql+mysqlconnector://{os.environ['USER_NAME']}:{os.environ['PASSWORD']}@{os.environ['PI']}/{os.environ['MAIN_DB']}" ) # Rasberry Pi engine
# lite_engine = create_engine(f"sqlite:///punk-records.db" )

twitch = Twitch(os.environ['TWITCH_APP_ID'], os.environ['TWITCH_APP_SECRET'])

In [12]:
Game_Meta.game_id

In [13]:
with Session(sql_engine) as session:
    print(session.query(Game_Meta.game_id, Game_Meta.game_name).where(Game_Meta.game_name.ilike('%Clash%')).all())

[(491168, 'Clash Royale')]


In [4]:
mobile_folder = os.environ['MOBILE_FOLDER']
highlights_folder = os.environ['PROCESSED_HIGHLIGHTS']
ml_folder = os.environ['ML_FOLDER']
assets = os.environ['ASSETS']

CLIENT_SECRET_FILE = 'client_secrets.json'
API_NAME = 'youtube'
API_VERSION = 'v3'
# recommend pulling as much access as possible since the Create_Service function will create a pickle token of your authentication allow single authentication for extended period.
SCOPES = ["https://www.googleapis.com/auth/youtube.upload", 'https://www.googleapis.com/auth/youtube']

# Code that initiates connection to the Google API - pulled from Google.py. Difference from using Google's built-in service is the following script creates convenience of storing pickle with necessary authentication
service = Create_Service(CLIENT_SECRET_FILE, API_NAME, API_VERSION, SCOPES)
with Session(sql_engine)as session:
        for clip_id, game_title, clip_creator, console, clip_title, video_name, clip_url in session.query(TwitchVideos.id, Game_Meta.game_name, TwitchVideos.creator_name, Game_Meta.platform, TwitchVideos.title, TwitchVideos.video_name, TwitchVideos.url).select_from(join(Game_Meta,TwitchVideos)).where(and_(or_(TwitchVideos.published == None, TwitchVideos == PublishingStatus.d), TwitchVideos.mobiles_videos_processed == True, TwitchVideos.video_type == TwitchVideoStyle.CLIP)).order_by(TwitchVideos.view_count.desc()).all()[:1]:
            # print(game_title, clip_creator, console, clip_title, video_name)
            # First upload the full screen
            request_body = {
                'snippet': {
                    'categoryId': 20,
                    'title': " | ".join([console, game_title+" Gameplay", clip_title]),
                    'description': createDescription(game_title, os.environ['YT_CHANNEL_ID'], clip_creator, console, os.environ['CHANNEL'], clip_url),
                    'tags': ["Video Games", game_title, console, "Gaming"]
                    },
                    'status': {
                        'privacyStatus': 'public',
                        'selfDeclaredMadeForKids': False, 
                        'madeForKids': False,
                    },
                    'notifySubscribers': True
                }
            # mediaFile = MediaFileUpload(os.path.join(fullscreen_folder, video_name))
            # normal_response_upload = service.videos().insert(
            #     part='snippet,status',
            #     body=request_body,
            #     media_body=mediaFile
            # ).execute()
            # # pause after upload
            # print("Normal Upload complete", clip_title)
            # time.sleep(5)
            # playlistId = session.query(PlayList_yt.id).where(PlayList_yt.title.ilike(f'%{game_title}%')).first()[0]
            # service.playlistItems().insert(part = "snippet", body = {"snippet": {'playlistId': playlistId, 'resourceId': {'videoId': normal_response_upload.get('id'), 'kind': "youtube#video"}}}).execute()

            #upload the short
            request_body['description'] = createDescription(game_title, os.environ['YT_CHANNEL_ID'], clip_creator, console, os.environ['CHANNEL'], clip_url, mode='shorts')
            mediaFile = MediaFileUpload(os.path.join(mobile_folder, video_name))
            mobile_response_upload = service.videos().insert(
                part='snippet,status',
                body=request_body,
                media_body=mediaFile
            ).execute()
            # time.sleep(5)
            # playlistId = session.query(PlayList_yt.id).where(PlayList_yt.title.ilike(f'%short%')).first()[0]
            # service.playlistItems().insert(part = 'snippet', body = {"snippet": {'playlistId': playlistId, 'resourceId': {'videoId': mobile_response_upload.get('id'), 'kind': "youtube#video"}}}).execute()
            # session.execute(update(TwitchVideos).where(TwitchVideos.id == clip_id).values({'published' : PublishingStatus.m}))
            # session.commit()
            print("Short Upload complete", clip_title)



client_secrets.json-youtube-v3-(['https://www.googleapis.com/auth/youtube.upload', 'https://www.googleapis.com/auth/youtube'],)
['https://www.googleapis.com/auth/youtube.upload', 'https://www.googleapis.com/auth/youtube']
youtube service created successfully
Short Upload complete Getting the Chicago Typewriter 


In [9]:
with Session(sql_engine) as session:
    for ix in session.execute("Show tables").all():
        print(ix)
    # session.execute('drop table yt_video_playlist_mapper')
    # session.execute('drop table yt_video_mapper')
    # session.execute('drop table yt_playlist_mapper')
    # session.execute('drop table yt_playlists')
    # session.execute('drop table clip_merits_tracker')
    # session.execute('drop table clip_audio')
    
    
    session.commit()

('advantage_balance_sheet',)
('advantage_cashflow',)
('advantage_earnings',)
('advantage_income_statements',)
('asset_prices',)
('asset_prices2',)
('balance_sheet_advantage',)
('broadcaster',)
('cashflow_advantage',)
('chatter_stats',)
('clip_tracker',)
('color_mapper',)
('color_mapper2',)
('country_codes',)
('earnings_advantage',)
('game_meta',)
('game_requests',)
('holiday_tracker',)
('income_statements_advantage',)
('interp_sov_yc',)
('interp_zc',)
('ledger',)
('meritocracy',)
('power_viewer',)
('real_gdp',)
('sov_yc',)
('spam_monitor',)
('stock_meta',)
('stock_prices',)
('stock_symbol_mapping',)
('story_book',)
('strawpoll_monitor',)
('the_big_house',)
('the_general_store',)
('thumbnail_mapper',)
('top_games',)
('twitch_schedule',)
('twitch_video_tracker',)
('twitch_viewers',)
('video_polled',)
('video_shown',)
('zerorates_yc',)


In [3]:
createDescription("Remnant", 'something', 'theroundwon', 'PS5', 'theroundwon', 'smeo', mode= generate_timestamps(sql_engine, ' Midnight Madness') )

'Follow me on Twitch for weekly live gameplay\nhttps://twitch.tv/theroundwon\nCheck out the original clip on Twitch! smeo\n\n00:00:09 - Onslaught\n00:02:57 - Vargr, The Warg\n00:07:39 - Gorefist\n00:09:41 - Riphide\n00:11:51 - Ixillis\n00:16:02 - Raze\n00:19:09 - Tian, The Assassin\n\n#Remnant #videogames #followme #twitch #casualplay #TheRoundWon #PS5\nGame play Footage from Remnant\nBig Thanks to theroundwon for capturing the footage!\n\nSubscribe to get notified of new videos:\nhttps://www.youtube.com/channel/something?sub_confirmation=1'

In [20]:
sub_query = select(TwitchVideos.title).group_by(TwitchVideos.title).having( func.count(TwitchVideos.title).label("video_count") > 1) # used as input in sub_query2
sub_query2 = select(TwitchVideos.title, TwitchVideos.created_at, TwitchVideos.video_type.label('video_type'), func.dense_rank().over(order_by=TwitchVideos.created_at, partition_by=[TwitchVideos.title, TwitchVideos.video_type]).label("counts") ).where(TwitchVideos.title.in_(sub_query)).subquery()

In [14]:
highlightModeIx = func.substring_index(midString, ' ', -1)
with Session(sql_engine) as session:

<sqlalchemy.sql.functions.Function at 0x7f9559fe2d60; substring_index>

In [23]:
title = "Midnight Prowl"
strtString = func.substring_index(TwitchVideos.title, "|", 1)
endString = func.substring_index(TwitchVideos.title, "|", -1)
input_highlights = os.environ['IN_PROCESS_HIGHLIGHTS']
output_highlights = os.environ['PROCESSED_HIGHLIGHTS']
midString = func.trim(func.substring(
    TwitchVideos.title,
    func.length(strtString)+2,
    func.length(TwitchVideos.title) - func.length(strtString) - func.length(endString)-2
))

highlightMode = func.substring_index(midString, ' ', 1)
highlightModeIx = func.substring_index(midString, ' ', -1)
with Session(sql_engine) as session:
    check_returns = session.query(TwitchVideos.id, Game_Meta.game_name, TwitchVideos.creator_name, Game_Meta.platform, TwitchVideos.title, endString, TwitchVideos.url).select_from(join(Game_Meta,TwitchVideos)).where(and_(or_(TwitchVideos.published == None, TwitchVideos == PublishingStatus.d), TwitchVideos.video_type == TwitchVideoStyle.HIGHLIGHT, endString.ilike(f"%{title}%"))).order_by(TwitchVideos.view_count.desc()).all()
    clip_id, game_title, clip_creator, console, clip_title, video_name, clip_url = check_returns[0]
    # for clip_id, game_title, clip_creator, console, clip_title, video_name, clip_url in check_returns:
    #     print(video_name)
        
    #     session.execute(update(TwitchVideos).where(TwitchVideos.id == clip_id).values({'published' : PublishingStatus.d}))
    #     session.commit()
        # print(createDescription(game_title, os.environ['YT_CHANNEL_ID'], clip_creator, console, os.environ['CHANNEL'], clip_url, mode=generate_timestamps(sql_engine, title)))

In [26]:
clip_id

'1781122370'

In [7]:
timestamps_to_string(
generate_timestamps(sql_engine, " The Hardest Boss is the server"))

'00:00:09 - Brudvaak, The Rider\n00:02:39 - Mangler\n00:04:18 - Scald & Sear\n00:06:58 - Claviger\n00:10:18 - The Thrail (aka "Predator Guy")\n00:12:35 - Obryk, The Shield Warden\n'

In [31]:
check_returns[:1]


createDescription(game_title, os.environ['YT_CHANNEL_ID'], clip_creator, console, os.environ['CHANNEL'], clip_url, mode=generate_timestamps(session_engine, title))

NameError: name 'session_engine' is not defined

In [22]:
/Users/avikchowdhury/Library/Mobile Documents/com~apple~CloudDocs/Twitch Resources/Twitch VODs/Processed Files/ The hardest boss is the server.mp4

[[]]

In [24]:
with Session(sql_engine) as session:
  check_returns = [session.query(TwitchVideos.id, Game_Meta.game_name, TwitchVideos.creator_name, Game_Meta.platform, TwitchVideos.title, TwitchVideos.video_name, TwitchVideos.url).select_from(join(Game_Meta,TwitchVideos)).where(and_(or_(TwitchVideos.published == None, TwitchVideos == PublishingStatus.d), TwitchVideos.mobiles_videos_processed == True, TwitchVideos.video_type == TwitchVideoStyle.CLIP, endString.ilike(f"%{title}"))).order_by(TwitchVideos.view_count.desc()).all()]

[]


In [ ]:
createDescription("Remnant", 'something', 'theroundwon', 'PS5', 'theroundwon', 'smeo', mode= generate_timestamps(sql_engine, ' Midnight Madness') )

NameError: name 'createDescription' is not defined

In [10]:
strtString = func.substring_index(TwitchVideos.title, "|", 1)
endString = func.substring_index(TwitchVideos.title, "|", -1)
input_highlights = os.environ['IN_PROCESS_HIGHLIGHTS']
output_highlights = os.environ['PROCESSED_HIGHLIGHTS']
midString = func.trim(func.substring(
    TwitchVideos.title,
    func.length(strtString)+2,
    func.length(TwitchVideos.title) - func.length(strtString) - func.length(endString)-2
))

highlightMode = func.substring_index(midString, ' ', 1)
highlightModeIx = func.substring_index(midString, ' ', -1)
with Session(sql_engine) as session:
    print(session.query(endString, func.sum(TwitchVideos.view_count)).group_by(endString).order_by(func.sum(TwitchVideos.view_count).desc()).where(TwitchVideos.video_type == TwitchVideoStyle.HIGHLIGHT).all())

[(' The hardest boss is the server', Decimal('23')), (' Path to Elden Lord', Decimal('22')), (' Midnight Prowl', Decimal('14')), (' Remnant Extras', Decimal('6')), (' Midnight Madness', None)]


In [40]:
with Session(sql_engine) as session:
   print(session.query(Game_Meta.game_id, Game_Meta.game_name).where(Game_Meta.downloaded == True).all())

[(6369, 'God of War'), (21812, "Demon's Souls"), (110758, 'The Legend of Zelda: Breath of the Wild'), (489401, 'Call of Duty: Black Ops 3'), (491168, 'Clash Royale'), (493997, 'Super Mario Odyssey'), (497057, 'Destiny 2'), (497388, 'A Way out'), (499856, 'Ghost of Tsushima'), (504461, 'Super Smash Bros. Ultimate'), (509066, 'Remnant: From the Ashes'), (512953, 'Elden Ring'), (518006, 'Stray'), (518015, 'Horizon Forbidden West'), (518019, 'Returnal'), (518213, 'It Takes Two'), (518847, 'Overcooked! All You Can Eat'), (350892884, "Ghost of Tsushima: Director's Cut")]


ffmpeg  -i "/Users/avikchowdhury/Library/Mobile Documents/com~apple~CloudDocs/Twitch Resources/Twitch Clips/Raw Clips/Entering Super Spirit Laundromat.mp4" -f lavfi -i color=size=1920x1080:duration=3:rate=60:color=black  -video_track_timescale 60k -c:a copy -shortest test_holder.mp4

ffmpeg -i "/Users/avikchowdhury/Library/Mobile Documents/com~apple~CloudDocs/Twitch Resources/Twitch Clips/Raw Clips/Entering Super Spirit Laundromat.mp4"


ffmpeg -i title_holder.mp4 -f lavfi -i anullsrc=r=48000:cl=stereo -c:v copy -c:a aac -shortest test.mp4


Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 'tmp_folder/Brudvaak, The Rider _ts.mp4':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2avc1mp41
    encoder         : Lavf59.27.100
  Duration: 00:00:03.07, start: 0.000000, bitrate: 54 kb/s
  Stream #0:0[0x1](und): Audio: aac (LC) (mp4a / 0x6134706D), 11025 Hz, mono, fltp, 0 kb/s (default)
    Metadata:
      handler_name    : SoundHandler
      vendor_id       : [0][0][0][0]
  Stream #0:1[0x2](und): Video: h264 (High) (avc1 / 0x31637661), yuv420p(progressive), 1920x1080 [SAR 1:1 DAR 16:9], 42 kb/s, 60 fps, 60 tbr, 15360 tbn (default)
    Metadata:
      handler_name    : VideoHandler
      vendor_id       : [0][0][0][0]
      encoder         : Lavc59.37.100 libx264



Input #0, mov,mp4,m4a,3gp,3g2,mj2, from '/Users/avikchowdhury/Library/Mobile Documents/com~apple~CloudDocs/Twitch Resources/Twitch VODs/In Process/Brudvaak, The Rider | BOSS 1 | The hardest boss is the server.mp4':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2avc1mp41
    title           : Brudvaak, The Rider | BOSS 1 | The hardest boss is the server
    artist          : TheRoundWon
    encoder         : Lavf59.27.100
  Duration: 00:02:27.03, start: 0.000000, bitrate: 6852 kb/s
  Stream #0:0[0x1](und): Audio: aac (LC) (mp4a / 0x6134706D), 48000 Hz, stereo, fltp, 129 kb/s (default)
    Metadata:
      handler_name    : SoundHandler
      vendor_id       : [0][0][0][0]
  Stream #0:1[0x2](und): Video: h264 (High) (avc1 / 0x31637661), yuv420p(progressive), 1920x1080, 6710 kb/s, 60 fps, 60 tbr, 15360 tbn (default)
    Metadata:
      handler_name    : VideoHandler
      vendor_id       : [0][0][0][0]
      encoder         : Lavc59.37.100 libx264

In [14]:
input1 = ffmpeg.input("/Users/avikchowdhury/Library/Mobile Documents/com~apple~CloudDocs/Twitch Resources/Twitch Clips/Raw Clips/Entering Super Spirit Laundromat.mp4")

In [15]:
input1.filter('color')

color()[None] <109857dab4f7>

In [40]:
audio = AudioSegment.silent(duration = 3000)
audio.export("title_audio.m4a", format='m4a')

CouldntEncodeError: Encoding failed. ffmpeg/avlib returned error code: 1

Command:['ffmpeg', '-y', '-f', 'wav', '-i', '/var/folders/rn/607w536s6bz6576kw_560yv40000gn/T/tmpzar5keqr', '-f', 'm4a', '/var/folders/rn/607w536s6bz6576kw_560yv40000gn/T/tmpw8r5p8p7']

Output from ffmpeg/avlib:

ffmpeg version 5.1.2 Copyright (c) 2000-2022 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/5.1.2_6 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx2

In [121]:
sub_query = select(TwitchVideos.title).group_by(TwitchVideos.title).having( func.count(TwitchVideos.title).label("video_count") > 1) # used as input in sub_query2
sub_query2 = select(TwitchVideos.title, TwitchVideos.created_at, TwitchVideos.video_type.label('video_type'), func.dense_rank().over(order_by=TwitchVideos.created_at, partition_by=[TwitchVideos.title, TwitchVideos.video_type]).label("counts") ).where(TwitchVideos.title.in_(sub_query)).subquery()

with Session(sql_engine) as session:
    adjustments = session.query(sub_query2).where(sub_query2.c.video_type == TwitchVideoStyle.HIGHLIGHT).all() #alter video names which had groupings of the same name

print(adjustments)

[]


In [40]:
with Session(sql_engine) as session:
    print(session.query(Clip_Tracker.yt_filename.distinct()).limit(5).all())

[(None,)]


In [4]:
strtString = func.substring_index(TwitchVideos.title, "|", 1)
endString = func.substring_index(TwitchVideos.title, "|", -1)
input_highlights = os.environ['IN_PROCESS_HIGHLIGHTS']
output_highlights = os.environ['PROCESSED_HIGHLIGHTS']
midString = func.trim(func.substring(
    TwitchVideos.title,
    func.length(strtString)+2,
    func.length(TwitchVideos.title) - func.length(strtString) - func.length(endString)-2
))

highlightMode = func.substring_index(midString, ' ', 1)
highlightModeIx = func.substring_index(midString, ' ', -1)


title_files = []
with Session(sql_engine) as session:
    for bossName, fil, end in  session.query(strtString, TwitchVideos.video_name, endString).where(TwitchVideos.video_type == TwitchVideoStyle.HIGHLIGHT).order_by(endString, highlightModeIx).where(highlightMode == 'BOSS').all():
        print(bossName, end.strip())

The Warden  Crash Landing
Dreameater  Crash Landing
Totem Father   Crash Landing
Tian, The Assassin   Crash Landing
Scourge   Crash Landing
The Ent   Crash Landing
Scald & Sear   Crash Landing
Shatter & Shade   Crash Landing
Totem Father   Crash Landing
Riphide  Friday Frenzy
Riphide  Friday Frenzy
Canker  Friday Frenzy
The Warden  Friday Frenzy
The Ent  Friday Frenzy
Scourge  Friday Frenzy
The Thrall  Friday Frenzy
Raze  Friday Frenzy
Ikro, the ice Conjurer  Friday Frenzy
Claviger  Friday Frenzy
Onslaught  Midnight Madness
Vargr, The Warg  Midnight Madness
Gorefist  Midnight Madness
Riphide  Midnight Madness
Ixillis  Midnight Madness
Raze  Midnight Madness
The Ent (Failure)  Midnight Prowl
Scald & Sear  Midnight Prowl
Raze   Midnight Prowl
Unclean One  Midnight Prowl
Barbed Terror  Midnight Prowl
Canker  Midnight Prowl
The Ent (Redemption)  Midnight Prowl
Vargr, The Warg  Midnight Prowl
Thrall (aka "The Predator Guy")  Midnight Prowl
Onslaught  Midnight Prowl
Putrid Avatar  Path to El

In [50]:
with Session(sql_engine) as session:
    print(session.query(func.distinct(strtString)).where(and_(TwitchVideos.video_type == TwitchVideoStyle.HIGHLIGHT, highlightMode == 'BLOOPER')).all())

[('Starting to cut it a little close ',)]


In [12]:
with Session(sql_engine) as session:
    titles = [ix[0]for ix in session.query(func.distinct(endString)).where(and_(TwitchVideos.video_type == TwitchVideoStyle.HIGHLIGHT, or_(TwitchVideos.published == None, TwitchVideos.published != PublishingStatus.d))).all()]

In [4]:
titles = unfinished_highlights(sql_engine)

In [15]:
unfinished

[' Friday Frenzy']

In [17]:
def ready_highlights(session_engine) -> list:
    titles = unfinished_highlights(sql_engine)
    with Session(session_engine) as session:
        unfinished = [ix[0]for ix in session.query(func.distinct(endString)).where(endString.in_(titles)).where(TwitchVideos.mobiles_videos_processed == False).all()]
        return [title for title in titles if title not in unfinished]

[' Crash Landing', ' Campfire Cookoff']


In [26]:
total_duration

4930.0

In [39]:
def convert_duration_to_timestamp(duration: float)-> str:
    """Convert duration of seconds into youtube friendly timestamp"""
    hours = int(duration//(60*60))
    duration -= hours*(60*60)
    minutes = int(duration // 60)
    duration -= minutes*(60)
    return f"{hours:02d}:{minutes:02d}:{int(duration):02d}"


In [53]:
timestamps

{'Remnant Extras': {'Starting to cut it a little close': '00:00:06'},
 'Path to Elden Lord': {'Putrid Avatar': '00:00:06',
  'Tibia Mariner': '00:03:37'},
 'The hardest boss is the server': {'Brudvaak, The Rider': '00:00:06',
  'Mangler': '00:02:36',
  'Scald & Sear': '00:04:15',
  'Claviger': '00:06:55',
  'The Thrail (aka "Predator Guy")': '00:10:15',
  'Obryk, The Shield Warden': '00:12:32'},
 'Midnight Prowl': {'The Ent (Failure)': '00:00:06',
  'Scald & Sear': '00:02:36',
  'Raze': '00:04:58',
  'Unclean One': '00:07:21',
  'Barbed Terror': '00:10:12',
  'Canker': '00:13:00',
  'The Ent (Redemption)': '00:16:05',
  'Vargr, The Warg': '00:19:36',
  'Thrall (aka "The Predator Guy")': '00:24:28',
  'Onslaught': '00:30:28'},
 'Midnight Madness': {'Onslaught': '00:00:06',
  'Vargr, The Warg': '00:02:54',
  'Gorefist': '00:07:36',
  'Riphide': '00:09:38',
  'Ixillis': '00:11:48',
  'Raze': '00:15:59',
  'Tian, The Assassin': '00:19:03'}}

In [56]:
generate_timestamps(sql_engine, ' Midnight Madness')

{'Onslaught': '00:00:06',
 'Vargr, The Warg': '00:02:54',
 'Gorefist': '00:07:36',
 'Riphide': '00:09:38',
 'Ixillis': '00:11:48',
 'Raze': '00:15:59',
 'Tian, The Assassin': '00:19:03'}

In [62]:
def generate_timestamps(session_engine, title, duration_offset = 3):
    substamps = {}
    total_duration = duration_offset
    title_files = []
    modes = ['MISSION', 'BOSS', 'FAIL', 'BLOOPER']
    with Session(session_engine) as session:
        for mode in modes:
            total_duration += 3
            if bool(session.query(strtString, TwitchVideos.video_name, endString).where(TwitchVideos.video_type == TwitchVideoStyle.HIGHLIGHT).order_by(endString, highlightModeIx).where(highlightMode == mode).where(endString == title).all()):

                for duration, fil, end in  session.query(TwitchVideos.duration, TwitchVideos.video_name, strtString).where(TwitchVideos.video_type == TwitchVideoStyle.HIGHLIGHT).order_by(endString, highlightModeIx).where(highlightMode == mode).where(endString == title).all():
                    substamps[end.strip()] = convert_duration_to_timestamp(total_duration)
                    total_duration += duration + 3
    return substamps

In [61]:
timestamps_to_string(generate_timestamps(sql_engine, ' Midnight Madness'))

'00:00:06 - Onslaught\n00:02:54 - Vargr, The Warg\n00:07:36 - Gorefist\n00:09:38 - Riphide\n00:11:48 - Ixillis\n00:15:59 - Raze\n00:19:03 - Tian, The Assassin\n'

In [60]:
def timestamps_to_string(timestamp: dict) -> str:
    """Convert dictionary of timestamps into string

    Args:
        timestamp (dict): dictionary of time stamps {'name': 'timestamp'}

    Returns:
        str: Youtube friendly timestamp string
    """
    output_str = ""
    for key, value in timestamps.items():
        output_str += value + f" - {key}\n"
    return output_str

In [52]:
modes = ['MISSION', 'BOSS', 'FAIL', 'BLOOPER']
total_duration = 0

with Session(sql_engine) as session:
    timestamps = {}
    for title in titles:
        substamps = {}
        total_duration = 6
        title_files = []
        for mode in modes:
            if bool(session.query(strtString, TwitchVideos.video_name, endString).where(TwitchVideos.video_type == TwitchVideoStyle.HIGHLIGHT).order_by(endString, highlightModeIx).where(highlightMode == mode).where(endString == title).all()):
    
                for duration, fil, end in  session.query(TwitchVideos.duration, TwitchVideos.video_name, strtString).where(TwitchVideos.video_type == TwitchVideoStyle.HIGHLIGHT).order_by(endString, highlightModeIx).where(highlightMode == mode).where(endString == title).all():
                    substamps[end.strip()] = convert_duration_to_timestamp(total_duration)
                    total_duration += duration + 3
        timestamps[title.strip()] = substamps



In [8]:
strtString = func.substring_index(TwitchVideos.title, "|", 1)
endString = func.substring_index(TwitchVideos.title, "|", -1)
input_highlights = os.environ['IN_PROCESS_HIGHLIGHTS']
output_highlights = os.environ['PROCESSED_HIGHLIGHTS']
midString = func.trim(func.substring(
    TwitchVideos.title,
    func.length(strtString)+2,
    func.length(TwitchVideos.title) - func.length(strtString) - func.length(endString)-2
))

highlightMode = func.substring_index(midString, ' ', 1)
highlightModeIx = func.substring_index(midString, ' ', -1)
with Session(sql_engine) as session:
    titles = [ix[0]for ix in session.query(func.distinct(endString)).where(TwitchVideos.video_type == TwitchVideoStyle.HIGHLIGHT).all()]
    titles = [f"{title}.mp4" for title in titles]
    completed_highlights = os.listdir(output_highlights)
    output = [title for title in titles if title not in completed_highlights]


In [3]:
unfinished_highlights(sql_engine)

NameError: name 'unfinished_highlights' is not defined

In [24]:
def make_highlight_segment(session_engine, title_files: list,  tmp_folder: str, mode: str) -> list:
    with Session(session_engine) as session:
        if bool(session.query(strtString, TwitchVideos.video_name, endString).where(TwitchVideos.video_type == TwitchVideoStyle.HIGHLIGHT).order_by(endString, highlightModeIx).where(highlightMode == mode).where(endString == title).all()):
            ffmpeg.concat(add_transition(create_title(mode+ " Highlights", 'title_holder.mp4')), a1, v=1, a=1).output(os.path.join(tmp_folder, f"{mode}_ts.mp4"), enc_time_base='1/60').run()
            mode_title_path = os.path.join(tmp_folder, f"{mode}_ts.mp4")
            title_files.append(f"'{mode_title_path}'")
            for bossName, fil, end in  session.query(strtString, TwitchVideos.video_name, endString).where(TwitchVideos.video_type == TwitchVideoStyle.HIGHLIGHT).order_by(endString, highlightModeIx).where(highlightMode == mode).where(endString == title).all():
                a1 = ffmpeg.input("title_holder.mp4").audio
                ffmpeg.concat(add_transition(create_title(bossName, 'title_holder.mp4')), a1, v=1, a=1).output(os.path.join(tmp_folder, f"{bossName}_ts.mp4"), enc_time_base='1/60').run()
                boss_title_path = os.path.join(tmp_folder, f"{bossName}_ts.mp4")
                title_files.append(f"'{boss_title_path}'")
                title_files.append(f"'{os.path.join(input_highlights, fil)}'")
        return title_files
        
        

In [25]:
def make_highlights_compilation(session_engine, output_highlights: str, modes: list = ['MISSION', 'BOSS', 'FAIL', 'BLOOPER']):
    tmp_folder = os.environ['TMP_FOLDER']
    titles = ready_highlights(session_engine)
    for title in titles:
        clear_tempfiles()
        title_files = []
        a1 = ffmpeg.input("title_holder.mp4").audio
        ffmpeg.concat(add_transition(create_title(title, 'title_holder.mp4')), a1, v=1, a=1).output(f"tmp_folder/{title}_ts.mp4", enc_time_base='1/60').run()
        main_title_path = os.path.join(tmp_folder, f"{title}_ts.mp4")
        title_files.append(f"'{main_title_path}'")
        for mode in modes:
            title_files = make_highlight_segment(session_engine, title_files, tmp_folder, mode)
        with open('concat.txt', 'w') as f:
            f.writelines([('file %s\n' % input_path) for input_path in title_files])
        ffmpeg.input('concat.txt', format='concat', safe=0).output(os.path.join(output_highlights, f'{title}.mp4'), c='copy', r=60).run()

In [18]:
strtString = func.substring_index(TwitchVideos.title, "|", 1)
endString = func.substring_index(TwitchVideos.title, "|", -1)
input_highlights = os.environ['IN_PROCESS_HIGHLIGHTS']
output_highlights = os.environ['PROCESSED_HIGHLIGHTS']
midString = func.trim(func.substring(
    TwitchVideos.title,
    func.length(strtString)+2,
    func.length(TwitchVideos.title) - func.length(strtString) - func.length(endString)-2
))

highlightMode = func.substring_index(midString, ' ', 1)
highlightModeIx = func.substring_index(midString, ' ', -1)

modes = ['MISSION', 'BOSS', 'FAIL', 'BLOOPER']
clear_tempfiles()
with Session(sql_engine) as session:
    titles = [ix[0]for ix in session.query(func.distinct(endString)).where(TwitchVideos.video_type == TwitchVideoStyle.HIGHLIGHT).all()]
with Session(sql_engine) as session:
    for title in titles:
        clear_tempfiles()
        title_files = []
        a1 = ffmpeg.input("title_holder.mp4").audio
        ffmpeg.concat(add_transition(create_title(title, 'title_holder.mp4')), a1, v=1, a=1).output(f"tmp_folder/{title}_ts.mp4", enc_time_base='1/60').run()
        title_files.append(f"'/Users/avikchowdhury/Programming/Twitch_App/tmp_folder/{title}_ts.mp4'")
        for mode in modes:
            if bool(session.query(strtString, TwitchVideos.video_name, endString).where(TwitchVideos.video_type == TwitchVideoStyle.HIGHLIGHT).order_by(endString, highlightModeIx).where(highlightMode == mode).where(endString == title).all()):
                ffmpeg.concat(add_transition(create_title(mode+ " Highlights", 'title_holder.mp4')), a1, v=1, a=1).output(f"tmp_folder/{mode}_ts.mp4", enc_time_base='1/60').run()
                title_files.append(f"'/Users/avikchowdhury/Programming/Twitch_App/tmp_folder/{mode}_ts.mp4'")
                for bossName, fil, end in  session.query(strtString, TwitchVideos.video_name, endString).where(TwitchVideos.video_type == TwitchVideoStyle.HIGHLIGHT).order_by(endString, highlightModeIx).where(highlightMode == mode).where(endString == title).all():
                    a1 = ffmpeg.input("title_holder.mp4").audio
                    ffmpeg.concat(add_transition(create_title(bossName, 'title_holder.mp4')), a1, v=1, a=1).output(f"tmp_folder/{bossName}_ts.mp4", enc_time_base='1/60').run()
                    title_files.append(f"'/Users/avikchowdhury/Programming/Twitch_App/tmp_folder/{bossName}_ts.mp4'")
                    title_files.append(f"'{os.path.join(input_highlights, fil)}'")
        with open('concat.txt', 'w') as f:
            f.writelines([('file %s\n' % input_path) for input_path in title_files])
        ffmpeg.input('concat.txt', format='concat', safe=0).output(os.path.join(output_highlights, f'{title}.mp4'), c='copy', r=60).run()


ffmpeg version 5.1.2 Copyright (c) 2000-2022 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/5.1.2_6 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack

In [9]:
os.path.join(output_highlights, f'{title}.mp4')

'/Users/avikchowdhury/Library/Mobile Documents/com~apple~CloudDocs/Twitch Resources/Twitch VODs/Processed Files/ Path to Elden Lord.mp4'

In [31]:
f"'{os.path.join(output_highlights, fil)}'"

"'/Users/avikchowdhury/Library/Mobile Documents/com~apple~CloudDocs/Twitch Resources/Twitch VODs/In Process/Scald & Sear | BOSS 3 | The hardest boss is the server.mp4'"

In [20]:
ffmpeg.input('concat.txt', format='concat', safe=0).output('output.mp4', c='copy', r=60).run()



ffmpeg version 5.1.2 Copyright (c) 2000-2022 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/5.1.2_6 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack

(None, None)

In [112]:
vid.size

[1920, 1080]

In [17]:
ffmpeg.filter_complex()

AttributeError: module 'ffmpeg' has no attribute 'filter_complex'

In [6]:

# ffmpeg.input("color=c=white:s=1280x720:r=30:d=1").drawtext(text="video1", x=10, y=10, fontsize=24, fontcolor='black')
# 



holder = ffmpeg.input("title_holder.mp4")


input1 = ffmpeg.input('video1.mp4')
input2 = ffmpeg.input('video2.mp4')

title1 = create_title('video1')
title2 = create_title('video2')

files = [holder, input1]

# eparate video and audio, then flat the array
video_and_audios_files = [item for sublist in map(lambda f: [f.video, f.audio], files) for item in sublist]

# concat all
joined = (
    ffmpeg
    .concat(*video_and_audios_files, v=1, a=1)
    .node
)

# merge video and audio
(
    ffmpeg
    .output(joined[0], joined[1], 'out.mp4')
    .run()
)

# add_transition(create_title("video1 test")).concat(input1).output('output.mp4')

# v = concatenated[0]
# a = concatenated[1]

# output = ffmpeg.


ffmpeg version 5.1.2 Copyright (c) 2000-2022 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/5.1.2_6 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack

Error: ffmpeg error (see stderr output for detail)

In [15]:
joined[-1]

concat(a=1, n=2, v=1)[-1] <100acc4d1e50>

In [38]:
concatenated.compile()

['ffmpeg',
 '-i',
 'title_holder.mp4',
 '-i',
 'video1.mp4',
 '-filter_complex',
 '[0]drawtext=fontcolor=white:fontsize=24:text=video1:x=(w-text_w)/2:y=(h-text_h)/2[s0];[s0][0:a][1][1:a]concat=a=2:n=1:v=2[s1]',
 '-map',
 '[s1]',
 'output.mp4']

In [11]:
concatenated

concat(n=2)[None] <13de319a5292>

In [1]:
def toncat(*streams, **kwargs):
    video_stream_count = kwargs.get('v', 1)
    audio_stream_count = kwargs.get('a', 0)
    print(video_stream_count, audio_stream_count)

In [2]:
toncat.__name__

'toncat'

In [95]:
title1 = create_title('video1')

TypeError: filter() missing 1 required positional argument: 'filter_name'

In [91]:
with Session(sql_engine) as session:
    for tid, video, created in  session.query(TwitchVideos.id, TwitchVideos.video_name, TwitchVideos.created_at).where(TwitchVideos.video_type == TwitchVideoStyle.HIGHLIGHT).order_by(endString, highlightModeIx).where(highlightMode == 'BOSS').all():
        print(tid, video, created)



1781122366 The Ent (Failure) | BOSS 1 | Midnight Prowl.mp4 2023-03-31 22:02:30
1781122373 Scald & Sear | BOSS 2  | Midnight Prowl.mp4 2023-03-31 22:02:30
1781122380 Raze  | BOSS 3 | Midnight Prowl.mp4 2023-03-31 22:02:31
1781122368 Unclean One | BOSS 4 | Midnight Prowl.mp4 2023-03-31 22:02:30
1781122381 Barbed Terror | BOSS 5 | Midnight Prowl.mp4 2023-03-31 22:02:31
1781122382 Canker | BOSS 6 | Midnight Prowl.mp4 2023-03-31 22:02:31
1781122367 The Ent (Redemption) | BOSS 7 | Midnight Prowl.mp4 2023-03-31 22:02:30
1781122369 Vargr, The Warg | BOSS 8 | Midnight Prowl.mp4 2023-03-31 22:02:30
1781122370 Thrall (aka "The Predator Guy") | BOSS 8 | Midnight Prowl.mp4 2023-03-31 22:02:30
1781122371 Onslaught | BOSS 9 | Midnight Prowl.mp4 2023-03-31 22:02:30
1742597164 Putrid Avatar | BOSS | Path to Elden Lord.mp4 2023-02-19 05:13:37
1742605733 Tibia Mariner | BOSS | Path to Elden Lord.mp4 2023-02-19 05:27:38
1779309855 Brudvaak, The Rider | BOSS 1 | The hardest boss is the server.mp4 2023-03-2

In [52]:
with Session(sql_engine) as session:
    print(session.query(TwitchVideos.id, TwitchVideos.title).where(TwitchVideos.id =='1779309856').all())

[('1779309856', 'Scald & Sear | BOSS 3 | The hardest boss is the server')]


In [93]:
returns = twitch.get_videos(user_id=os.environ['TWITCH_CHANNEL_ID'], video_type = VideoType.HIGHLIGHT)
returns['data'] = [{**ix, **{'game_id': find_game_id(ix)}} for ix in returns['data']]

In [3]:
twitch.get_videos(user_id=os.environ['TWITCH_CHANNEL_ID'], video_type = VideoType.HIGHLIGHT)
seed = twitch.get_videos(user_id=os.environ['TWITCH_CHANNEL_ID'], video_type = VideoType.HIGHLIGHT)
seed['data'] = [{**ix, **{'game_id': find_game_id(ix), 'duration': convert_duration_to_sec(ix['duration'])}} for ix in seed['data']]
updateTables(sql_engine, seed, twitch, TwitchVideoStyle.HIGHLIGHT)

did this
did this
did this
did this
did this
did this
did this
did this
did this
did this
did this
did this
did this
did this
did this
did this
did this
did this
did this


In [47]:
video_mode = TwitchVideoStyle.HIGHLIGHT
with Session(sql_engine) as session:
    for item in adjust_output(seed, TwitchVideos):

        if not bool(session.query(TwitchVideos).where(TwitchVideos.id==item['id']).all()):
            session.add(TwitchVideos(**item))
            session.commit()
        else:
            session.execute(update(TwitchVideos).where(TwitchVideos.id == item['id']).values(view_count= item['view_count'], title=item['title'], video_type=video_mode))
            session.commit()
            print('done', item['title'])


done Barbed Terror : BOSS 5 : Midnight Prowl
done Raze  : BOSS 3 : Midnight Prowl
done Canker : BOSS 6 : Midnight Prowl
done The Ent (Redemption) : BOSS 7 : Midnight Prowl
done The Ent (Failure) : BOSS 1 : Midnight Prowl
done Vargr, The Warg | BOSS 8 | Midnight Prowl
done Thrall (aka "The Predator Guy") | BOSS 8 | Midnight Prowl
done Scald & Sear | BOSS 2  | Midnight Prowl
done Onslaught | BOSS 9 | Midnight Prowl
done Unclean One | BOSS 4 | Midnight Prowl
done Claviger | BOSS 4 | The hardest boss is the server
done Scald & Sear | BOSS 3 | The hardest boss is the server
done Obryk, The Shield Warden | BOSS 6 | The hardest boss is the server
done Brudvaak, The Rider | BOSS 1 | The hardest boss is the server
done The Thrail (aka "Predator Guy") | BOSS 5 | The hardest boss is the server
done Mangler | BOSS 2 | The hardest boss is the server
done Tibia Mariner | BOSS | Path to Elden Lord
done Putrid Avatar | BOSS | Path to Elden Lord
done Starting to cut it a little close | Blooper | Remnan

In [33]:
item['title']

'Barbed Terror : BOSS 5 : Midnight Prowl'

In [95]:
returns['data'][8]['duration']

'6m26s'

In [96]:
hour_test = '1h4m8s'

In [103]:
convert_duration_to_sec(hour_test)

3848

In [105]:
{**{'test': 1}, **{'test': 2}}

{'test': 2}

In [102]:
def convert_duration_to_sec(duration):
    hour_splits = duration.split('h')
    hours = 0
    minutes = 0
    if len(hour_splits) > 1:
        hours = int(hour_splits[0]) * 60 * 60
        duration = hour_splits[-1]
    minute_splits = duration.split('m')
    if len(minute_splits) > 1:
        minutes = int(minute_splits[0]) * 60
        duration = minute_splits[-1]
    seconds = int(duration.split('s')[0])

    return hours+minutes+seconds
    


In [99]:
hour_test.split('d')[0]

'1h4m8s'

In [91]:
for ix in returns['data']:
    try:
        find_game_id(ix)
    except:
        print(ix)


In [62]:
returns['description'].strip()

'game_id: 509066\nCOOP: lmperialEmpire, Kaiser_39'

In [66]:
find_game_id(returns)

509066

In [75]:
find_coop(returns)

['lmperialEmpire', 'Kaiser_39']

In [74]:
def find_coop(returns):
    patterns = "COOP: .*"
    coops = re.findall(patterns, returns['description'])[0]
    return [x.strip() for x in coops.split(':')[-1].split(',')]

In [65]:
def find_game_id(returns):
    patterns = "game_id: \d*"
    game_id = re.findall(patterns, returns['description'])[0]
    return int(game_id.split(':')[-1].strip())

In [50]:
twitch.get_channel_information(os.environ['TWITCH_CHANNEL_ID'])

{'data': [{'broadcaster_id': '469646515',
   'broadcaster_login': 'theroundwon',
   'broadcaster_name': 'TheRoundWon',
   'broadcaster_language': 'en',
   'game_id': '491168',
   'game_name': 'Clash Royale',
   'title': 'Clash With Triangle',
   'delay': 0,
   'tags': ['English']}]}

In [48]:
twitch.get

{'data': []}

In [29]:
with Session(sql_engine) as session:
    session.execute("DROP table tag_stream_association")
    session.execute("DROP table tag_mapper")
    session.execute("DROP table stream_capture")
    session.execute("DROP table yt_video_playlist_mapper")
    session.execute("DROP Table yt_videos")

In [6]:
with Session(sql_engine) as session:
    print(session.query(TwitchVideos).limit(5).all())

[<StreamMaster.TwitchVideos object at 0x7fca19c98dc0>, <StreamMaster.TwitchVideos object at 0x7fca19ca5040>, <StreamMaster.TwitchVideos object at 0x7fca19ca50d0>, <StreamMaster.TwitchVideos object at 0x7fca19ca5130>, <StreamMaster.TwitchVideos object at 0x7fca19ca5190>]


In [31]:
with Session(sql_engine) as session:
    for tabl in session.execute("SHOW Tables").all():
        print(tabl)

('advantage_balance_sheet',)
('advantage_cashflow',)
('advantage_earnings',)
('advantage_income_statements',)
('asset_prices',)
('asset_prices2',)
('balance_sheet_advantage',)
('broadcaster',)
('cashflow_advantage',)
('chatter_stats',)
('clip_audio',)
('clip_merits_tracker',)
('clip_tracker',)
('color_mapper',)
('color_mapper2',)
('country_codes',)
('earnings_advantage',)
('game_meta',)
('game_requests',)
('holiday_tracker',)
('income_statements_advantage',)
('interp_sov_yc',)
('interp_zc',)
('ledger',)
('meritocracy',)
('power_viewer',)
('real_gdp',)
('sov_yc',)
('spam_monitor',)
('stock_meta',)
('stock_prices',)
('stock_symbol_mapping',)
('story_book',)
('strawpoll_monitor',)
('the_big_house',)
('the_general_store',)
('thumbnail_mapper',)
('top_games',)
('twitch_schedule',)
('twitch_video_tracker',)
('twitch_viewers',)
('video_polled',)
('video_shown',)
('yt_playlist_mapper',)
('yt_playlists',)
('yt_video_mapper',)
('yt_video_playlist_mapper',)
('zerorates_yc',)


In [8]:
distinct()

TypeError: distinct() missing 1 required positional argument: 'expr'

In [11]:
with Session(sql_engine) as session:
    # for table in session.execute("SHOW Tables").all():
    print(session.query(func.count(Clip_Tracker.title.distinct())).where(~Clip_Tracker.title.in_([clip[0]for clip in session.query(Clips.title).all()])).all())
        # print(table)


[(0,)]


In [ ]:
mysql_engine = create_engine(f"mysql+mysqlconnector://{os.environ['USER_NAME']}:{os.environ['PASSWORD']}@{os.environ['PI']}/{os.environ['MAIN_DB']}" )


clips_folder = os.environ['CLIPS_FOLDER']
mobile_folder = os.environ['MOBILE_FOLDER']
fullscreen_folder = os.environ['FULLSCREEN_FOLDER']
ml_folder = os.environ['ML_FOLDER']
assets = os.environ['ASSETS']

CLIENT_SECRET_FILE = 'client_secrets.json'
# YT_SECRET_FILE = 'client_secrets_yt.json'
API_NAME = 'youtube'
API_VERSION = 'v3'
# recommend pulling as much access as possible since the Create_Service function will create a pickle token of your authentication allow single authentication for extended period.
SCOPES = ["https://www.googleapis.com/auth/youtube.upload", 'https://www.googleapis.com/auth/youtube']

# Code that initiates connection to the Google API - pulled from Google.py. Difference from using Google's built-in service is the following script creates convenience of storing pickle with necessary authentication
service = Create_Service(CLIENT_SECRET_FILE, API_NAME, API_VERSION, SCOPES)

In [ ]:
os.environ['OAUTHLIB_INSECURE_TRANSPORT'] = '1'

In [2]:
twitch = Twitch(os.environ['TWITCH_APP_ID'], os.environ['TWITCH_APP_SECRET'])

In [ ]:
twitch.get_videos()

In [ ]:
VideoType.

In [22]:
twitch.get_videos( user_id = os.environ['TWITCH_CHANNEL_ID'], video_type=VideoType.HIGHLIGHT)

{'data': [{'id': '1781122381',
   'stream_id': '1',
   'user_id': '469646515',
   'user_login': 'theroundwon',
   'user_name': 'TheRoundWon',
   'title': 'Barbed Terror : Survivor Trio : Midnight Prowl',
   'description': '',
   'created_at': '2023-03-31T22:02:31Z',
   'published_at': '2023-03-31T22:02:31Z',
   'url': 'https://www.twitch.tv/videos/1781122381',
   'thumbnail_url': 'https://static-cdn.jtvnw.net/cf_vods/d2nvs31859zcd8/b9232fdbeb036c8a1f44_theroundwon_69720515881_4095389357//thumb/thumb1781122381-%{width}x%{height}.jpg',
   'viewable': 'public',
   'view_count': 1,
   'language': 'en',
   'type': <VideoType.HIGHLIGHT: 'highlight'>,
   'duration': '2m45s',
   'muted_segments': None},
  {'id': '1781122380',
   'stream_id': '1',
   'user_id': '469646515',
   'user_login': 'theroundwon',
   'user_name': 'TheRoundWon',
   'title': 'Raze  : Survivor Trio : Midnight Prowl',
   'description': '',
   'created_at': '2023-03-31T22:02:31Z',
   'published_at': '2023-03-31T22:02:31Z',
 

In [24]:
twitch.get_clips( os.environ['TWITCH_CHANNEL_ID'])

{'data': [{'id': 'CautiousProtectiveRabbitDAESuppy-vFBGb3gytYPINuHU',
   'url': 'https://clips.twitch.tv/CautiousProtectiveRabbitDAESuppy-vFBGb3gytYPINuHU',
   'embed_url': 'https://clips.twitch.tv/embed?clip=CautiousProtectiveRabbitDAESuppy-vFBGb3gytYPINuHU',
   'broadcaster_id': '469646515',
   'broadcaster_name': 'TheRoundWon',
   'creator_id': '53728939',
   'creator_name': 'FeedMeACat',
   'video_id': '',
   'game_id': '21812',
   'language': 'en',
   'title': 'I mean...',
   'view_count': 279,
   'created_at': '2022-01-27T05:40:45Z',
   'thumbnail_url': 'https://clips-media-assets2.twitch.tv/AT-cm%7CYhFwEbxQRm94MrWDyDbh1A-preview-480x272.jpg',
   'duration': 16.8,
   'vod_offset': None},
  {'id': 'AggressiveTacitLorisArgieB8-Q0BI2b7jLk54H1Ug',
   'url': 'https://clips.twitch.tv/AggressiveTacitLorisArgieB8-Q0BI2b7jLk54H1Ug',
   'embed_url': 'https://clips.twitch.tv/embed?clip=AggressiveTacitLorisArgieB8-Q0BI2b7jLk54H1Ug',
   'broadcaster_id': '469646515',
   'broadcaster_name': 'T

In [23]:
twitch.get_videos(user_id = os.environ['TWITCH_CHANNEL_ID'], video_type = VideoType.HIGHLIGHT, after ='eyJiIjpudWxsLCJhIjp7Ik9mZnNldCI6MjB9fQ')

{'data': [{'id': '1316171201',
   'stream_id': '1',
   'user_id': '469646515',
   'user_login': 'theroundwon',
   'user_name': 'TheRoundWon',
   'title': 'Evening Clash with Triangle',
   'description': '',
   'created_at': '2022-06-02T23:20:09Z',
   'published_at': '2022-06-02T23:20:09Z',
   'url': 'https://www.twitch.tv/videos/1316171201',
   'thumbnail_url': 'https://static-cdn.jtvnw.net/cf_vods/d2nvs31859zcd8/efec49da5ae4be32ba50_theroundwon_99652514227_4580910530/thumb/custom-db055fbb-5ab2-4d9f-8360-f0de07b8ab0f-%{width}x%{height}.jpeg',
   'viewable': 'public',
   'view_count': 1,
   'language': 'en',
   'type': <VideoType.HIGHLIGHT: 'highlight'>,
   'duration': '36m53s',
   'muted_segments': None},
  {'id': '1457456463',
   'stream_id': '1',
   'user_id': '469646515',
   'user_login': 'theroundwon',
   'user_name': 'TheRoundWon',
   'title': "Homie's got some legs",
   'description': '',
   'created_at': '2022-04-16T04:47:30Z',
   'published_at': '2022-04-16T04:47:30Z',
   'url'

In [ ]:
color_sample = px.colors.sequential.YlOrBr + px.colors.sequential.ice
random.seed(333)
random.shuffle(color_sample)
game_colors = {}
games = {}
cnt = 0
with Session(mysql_engine) as session:
    for game_id, game_name in session.query(Game_Meta.game_id,func.max(Game_Meta.game_name)).select_from(join(Game_Meta,Clip_Tracker)).group_by(Game_Meta.game_id).all():
        games[game_id] = game_name 
        game_colors[game_name] = color_sample[cnt]
        cnt += 1

In [ ]:
color_sample = px.colors.sequential.Viridis

with Session(mysql_engine) as session:
    creators = {creator[0]: color_sample[i] for i, creator in enumerate(session.query(distinct(Clip_Tracker.creator_name)).all())}

In [ ]:
# Build sa visual for un-published clips
data_seed = {}
with Session(mysql_engine) as session:
    # for url in session.query(Clip_Tracker.url).where(Clip_Tracker.published == None).order_by(Clip_Tracker.view_count.desc()).all()[:2]:
    for title, game_name, created_date, view_count in session.query(Clip_Tracker.title,Game_Meta.game_name, Clip_Tracker.created_at,  Clip_Tracker.view_count).select_from(join(Clip_Tracker, Game_Meta)).where(Clip_Tracker.published == None).order_by(Clip_Tracker.view_count.desc()).all():
        if bool(data_seed.get(game_name)):
            try:
                data_seed[game_name]['x'].append(date(created_date.year, created_date.month, created_date.day))
                data_seed[game_name]['y'].append(view_count)
                data_seed[game_name]['text'].append(title)
            except:
                data_seed[game_name]['x'] = [created_date]
                data_seed[game_name]['y'] = [view_count]
                data_seed[game_name]['color'] = game_colors[game_name]
        else:
            data_seed[game_name] = {'x': [], 'y': [], 'text': [] }
            data_seed[game_name]['x'].append(date(created_date.year, created_date.month, created_date.day))
            data_seed[game_name]['y'].append(view_count)
            data_seed[game_name]['text'].append(title)
            data_seed[game_name]['color'] = game_colors[game_name]
    # print(session.query(distinct(Clip_Tracker.game_id), Game_Meta.game_name).select_from(join(Clip_Tracker, Game_Meta)).where(Clip_Tracker.published == None).order_by(Clip_Tracker.view_count.desc()).all())
        # print(url[0])

data = []
for key, value in data_seed.items():
    data.append(go.Bar(x=value['x'], y = value['y'], text = value['text'], name=key, marker={'color': value['color']}))
go.Figure(data, go.Layout(barmode='stack', title={'text': "Performance of Unpublished Clips"}))

In [ ]:
# Build sa visual for Published clips
data_seed = {}
with Session(mysql_engine) as session:
    # for url in session.query(Clip_Tracker.url).where(Clip_Tracker.published == None).order_by(Clip_Tracker.view_count.desc()).all()[:2]:
    for title, game_name, created_date, view_count in session.query(Clip_Tracker.title,Game_Meta.game_name, Clip_Tracker.created_at,  Clip_Tracker.view_count).select_from(join(Clip_Tracker, Game_Meta)).where(Clip_Tracker.published != None).order_by(Clip_Tracker.view_count.desc()).all():
        if bool(data_seed.get(game_name)):
            try:
                data_seed[game_name]['x'].append(date(created_date.year, created_date.month, created_date.day))
                data_seed[game_name]['y'].append(view_count)
                data_seed[game_name]['text'].append(title)
            except:
                data_seed[game_name]['x'] = [created_date]
                data_seed[game_name]['y'] = [view_count]
        else:
            data_seed[game_name] = {'x': [], 'y': [], 'text': []}
            data_seed[game_name]['x'].append(date(created_date.year, created_date.month, created_date.day))
            data_seed[game_name]['y'].append(view_count)
            data_seed[game_name]['text'].append(title)
            data_seed[game_name]['color'] = game_colors[game_name]

data = []
for key, value in data_seed.items():
    data.append(go.Bar(x=value['x'], y = value['y'], text = value['text'], name=key,  marker={'color': value['color']}))
go.Figure(data, go.Layout(barmode='stack', title={'text': "Performance of Published Clips"}))
    # print(session.query(distinct(Clip_Tracker.game_id), Game_Meta.game_name).select_from(join(Clip_Tracker, Game_Meta)).where(Clip_Tracker.published == None).order_by(Clip_Tracker.view_count.desc()).all())
        # print(url[0])



In [ ]:
# Build sa visual for un-published clips
data_seed = {}
category_mapper = {'m': "Mobile Only", 'f': "Fully Published", "d": 'Desktop Only'}
with Session(mysql_engine) as session:
    # for url in session.query(Clip_Tracker.url).where(Clip_Tracker.published == None).order_by(Clip_Tracker.view_count.desc()).all()[:2]:
    for title, game_name, view_count, publishing_status, created_date in session.query(Clip_Tracker.title,Game_Meta.game_name, Clip_Tracker.view_count, Clip_Tracker.published, Clip_Tracker.created_at).select_from(join(Clip_Tracker, Game_Meta)).order_by(Clip_Tracker.view_count.desc()).all():
        if isinstance(publishing_status, PublishingStatus):
            publshing_status_name = category_mapper[publishing_status.name]
        else:
            if publishing_status != None:
                print(publishing_status)
            publshing_status_name = "Not Published"
        if bool(data_seed.get(publshing_status_name)):
            data_seed[publshing_status_name]['y'].append(game_name)
            data_seed[publshing_status_name]['x'].append(view_count)
            data_seed[publshing_status_name]['text'].append(title)
        else:
            data_seed[publshing_status_name] = {'x': [], 'y': [], 'text': []}
            data_seed[publshing_status_name]['x'].append(view_count)
            data_seed[publshing_status_name]['y'].append(game_name)
            data_seed[publshing_status_name]['text'].append(title)


    # print(session.query(distinct(Clip_Tracker.game_id), Game_Meta.game_name).select_from(join(Clip_Tracker, Game_Meta)).where(Clip_Tracker.published == None).order_by(Clip_Tracker.view_count.desc()).all())
        # print(url[0])

data = []

odered_list = ['Fully Published', "Mobile Only", 'Desktop Only', "Not Published"]
# for key, value in data_seed.items():
for key in odered_list:
    category_name = key
    data.append(go.Bar(x=data_seed[key]['x'], y = data_seed[key]['y'], text = data_seed[key]['text'], name= category_name, orientation='h'))
go.Figure(data, go.Layout(barmode='stack', title={'text': "Performance By Game and Publishing Status"}, xaxis = {'title': {'text': "View Count"}},yaxis={'categoryorder': 'total ascending', 'title': {'text': 'Game Title'}}))

In [ ]:
data = []

fig = make_subplots(rows=1, cols = 4, subplot_titles = ("Average Views per Clip", "Max Views for Clip", "Total Views of Clips", "Number of Clips"), shared_yaxes=True)
with Session(mysql_engine) as session:
    # for title, game_name, view_count, publishing_status in session.query(Clip_Tracker.title,Game_Meta.game_name, Clip_Tracker.view_count, Clip_Tracker.published).select_from(join(Clip_Tracker, Game_Meta)).order_by(Clip_Tracker.view_count.desc()).all():
    cnt = 0
    for game_name, avg_cnt, max_cnt, sum_cnt, video_cnt in session.query(Game_Meta.game_name, func.avg(Clip_Tracker.view_count), func.max(Clip_Tracker.view_count), func.sum(Clip_Tracker.view_count), func.count(Clip_Tracker.title)).select_from(join(Game_Meta, Clip_Tracker)).group_by(Clip_Tracker.game_id).order_by(func.sum(Clip_Tracker.view_count).desc()).all():
        fig.add_trace(go.Bar(x=[avg_cnt], y = [game_name], orientation='h', showlegend=False, marker= dict(color = [game_colors[game_name]], coloraxis='coloraxis')),row=1, col=1)
        fig.add_trace(go.Bar(x=[max_cnt], y = [game_name], orientation='h', showlegend=False, marker= dict(color = [game_colors[game_name]], coloraxis='coloraxis')),row = 1, col=2)
        fig.add_trace(go.Bar(x=[sum_cnt], y = [game_name], orientation='h', showlegend=False, marker= dict(color = [game_colors[game_name]], coloraxis='coloraxis')), row = 1, col=3)
        fig.add_trace(go.Bar(x=[video_cnt], y = [game_name], orientation='h', showlegend=False, marker= dict(color = [game_colors[game_name]], coloraxis='coloraxis')), row=1,  col=4)
        # # data.append(go.Bar(x=[avg_cnt], y = [game_name]))
        cnt +=1
fig.update_yaxes(type='category')
fig.update_layout(barmode='group', yaxis={'categoryorder': 'category descending'}, title={'text': "Breaking each game by different statistics"})
fig.show()
        

In [ ]:
playlists = service.playlists().list(part='id,snippet,status', channelId=os.environ['YT_CHANNEL_ID']).execute()
nextPage = playlists['nextPageToken']
while nextPage != None:
    seedPlaylist = service.playlists().list(part='id,snippet,status', channelId=os.environ['YT_CHANNEL_ID'], pageToken = nextPage).execute()
    playlists['items'] = playlists['items'] + seedPlaylist['items']
    if bool(seedPlaylist.get('nextPageToken')):
        nextPage = seedPlaylist['nextPageToken']
    else:
        nextPage = None

In [ ]:
SCOPES = ['https://www.googleapis.com/auth/yt-analytics.readonly']

API_SERVICE_NAME = 'youtubeAnalytics'
API_VERSION = 'v2'
CLIENT_SECRETS_FILE = 'client_secrets_desktop.json'

def get_service():
    flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
    credentials = flow.run_console()
    return build(API_SERVICE_NAME, API_VERSION, credentials = credentials)


def execute_api_request(client_library_function, **kwargs):
  response = client_library_function(
    **kwargs
  ).execute()

  return response
youtubeAnalytics = Create_Service(CLIENT_SECRETS_FILE, API_SERVICE_NAME, API_VERSION, SCOPES)
# if __name__ == '__main__':
#   # Disable OAuthlib's HTTPs verification when running locally.
#   # *DO NOT* leave this option enabled when running in production.
#   os.environ['OAUTHLIB_INSECURE_TRANSPORT'] = '1'

# youtubeAnalytics = get_service()

In [ ]:
video_metrics

In [ ]:
video_metrics = execute_api_request(
      youtubeAnalytics.reports().query,
      ids='channel==MINE',
      startDate='2022-03-01',
      endDate='2022-07-12',
      maxResults = 140,
      metrics='estimatedMinutesWatched,views,likes,dislikes,subscribersGained,subscribersLost',
      dimensions='video',
      sort='-estimatedMinutesWatched'
  )

In [ ]:
video_seed = {'video_id': [], 'style': [],'game_name': [], 'title': [], 'creator': [], 'view_count': []}
with Session(mysql_engine) as session:
    for video_id, style, game_name,  title, creator,  view_count in session.query(Video_yt.id, Video_yt.style, Game_Meta.game_name, Clip_Tracker.title, Clip_Tracker.creator_name, Clip_Tracker.view_count ).select_from(join(join(Clip_Tracker, Game_Meta), Video_yt)).where(Clip_Tracker.published != None).all():
        video_seed['video_id'].append(video_id)
        video_seed['style'].append(style.name)
        video_seed['game_name'].append(game_name)
        video_seed['title'].append(title)
        video_seed['creator'].append(creator)
        video_seed['view_count'].append(view_count)


In [ ]:
clip_metrics_df = pd.DataFrame(video_metrics['rows'], columns=['video_id', 'estimatedMinutesWatched', 'views', 'likes', 'dislikes', 'subscribersGained', 'subscribersLost']).set_index('video_id').join(pd.DataFrame(video_seed).set_index('video_id'))

In [ ]:
video_metrics = execute_api_request(
        youtubeAnalytics.reports().query,
        ids='channel==MINE',
        startDate='2022-04-01',
        endDate='2022-07-27',
        maxResults = 140,
        metrics='estimatedMinutesWatched,views,likes,dislikes,subscribersGained,subscribersLost',
        dimensions='video',
        sort='-estimatedMinutesWatched'
    )

video_seed = {'video_id': [], 'style': [],'game_name': [], 'title': [], 'creator': [], 'view_count': []}
with Session(mysql_engine) as session:
    for video_id, style, game_name,  title, creator,  view_count in session.query(Video_yt.id, Video_yt.style, Game_Meta.game_name, Clip_Tracker.title, Clip_Tracker.creator_name, Clip_Tracker.view_count ).select_from(join(join(Clip_Tracker, Game_Meta), Video_yt)).where(Clip_Tracker.published != None).all():
        video_seed['video_id'].append(video_id)
        video_seed['style'].append(style.name)
        video_seed['game_name'].append(game_name)
        video_seed['title'].append(title)
        video_seed['creator'].append(creator)
        video_seed['view_count'].append(view_count)

In [ ]:
video_metrics

In [ ]:
# video_seed, video_metrics = build_video_seed(youtubeAnalytics)
clip_metrics_df = pd.DataFrame(video_metrics['rows'], columns=['video_id', 'estimatedMinutesWatched', 'views', 'likes', 'dislikes', 'subscribersGained', 'subscribersLost']).set_index('video_id').join(pd.DataFrame(video_seed).set_index('video_id'))
clip_metrics_df['likes_ratio'] = (clip_metrics_df['likes']/(clip_metrics_df['likes'] + clip_metrics_df['dislikes'])).fillna(0)

In [ ]:
sub_group = clip_metrics_df.groupby('title').min()
game_sub = sub_group.loc[sub_group['game_name'] == "God of War"]

In [ ]:
clip_metrics_df.head()

In [ ]:
clip_metrics_df.dropna(axis=0, how= 'any', inplace=True)
clip_metrics_df.loc[clip_metrics_df.title.str.contains('Heroes')]

In [ ]:
video_metrics = execute_api_request(
        youtubeAnalytics.reports().query,
        ids='channel==MINE',
        startDate='2022-03-01',
        endDate='2022-07-12',
        maxResults = 140,
        metrics='estimatedMinutesWatched,views,likes,dislikes,subscribersGained,subscribersLost',
        dimensions='video',
        sort='-estimatedMinutesWatched'
    )

In [ ]:
video_metrics

In [ ]:
fig = make_subplots(rows=1, cols=2, subplot_titles = ("Twitch Views", "Youtube Shorts Views"), shared_yaxes=True)

for ix in clip_metrics_df.loc[(clip_metrics_df['game_name'].notnull()) & (clip_metrics_df['style'] == 'm'), :].index:
    fig.add_trace(go.Bar(x=[clip_metrics_df.loc[ix, 'view_count']], y= [clip_metrics_df.loc[ix, 'game_name']], name=clip_metrics_df.loc[ix, 'title'], text=[clip_metrics_df.loc[ix, 'title']], orientation='h', marker={'color': [clip_metrics_df.loc[ix, 'view_count']], 'coloraxis': 'coloraxis'}), row=1, col=1)
    fig.add_trace(go.Bar(x=[clip_metrics_df.loc[ix, 'views']], y= [clip_metrics_df.loc[ix, 'game_name']], name=clip_metrics_df.loc[ix, 'title'], text=[clip_metrics_df.loc[ix, 'title']], orientation='h', marker={'color': [clip_metrics_df.loc[ix, 'view_count']], 'coloraxis': 'coloraxis'}), row=1, col=2)

fig.update_layout(height = 400, barmode='stack', title={'text': "Published Shorts vs Clips "}, yaxis2 = {'categoryorder': "total ascending"} )
fig.update_coloraxes(showscale=False)
fig.show()


In [ ]:
fig.layout.height

In [ ]:
fig = make_subplots(rows=1, cols=2, subplot_titles = ("Twitch Views", "Youtube Full Screen Views"), shared_yaxes=True)

for ix in clip_metrics_df.loc[(clip_metrics_df['game_name'].notnull()) & (clip_metrics_df['style'] == 'f'), :].index:
    fig.add_trace(go.Bar(x=[clip_metrics_df.loc[ix, 'view_count']], y= [clip_metrics_df.loc[ix, 'game_name']], name=clip_metrics_df.loc[ix, 'title'], text=[clip_metrics_df.loc[ix, 'title']], orientation='h', marker={'color': [clip_metrics_df.loc[ix, 'view_count']], 'coloraxis': 'coloraxis'}), row=1, col=1)
    fig.add_trace(go.Bar(x=[clip_metrics_df.loc[ix, 'views']], y= [clip_metrics_df.loc[ix, 'game_name']], name=clip_metrics_df.loc[ix, 'title'], text=[clip_metrics_df.loc[ix, 'title']], orientation='h', marker={'color': [clip_metrics_df.loc[ix, 'views']], 'coloraxis': 'coloraxis'}), row=1, col=2)

fig.update_layout(barmode='stack', title={'text': "Published Full Screens vs Clips "})
fig.update_coloraxes(showscale=False)
fig.show()


In [ ]:
clip_metrics_df['likes_ratio'] = (clip_metrics_df['likes']/(clip_metrics_df['likes'] + clip_metrics_df['dislikes'])).fillna(0)

In [ ]:
game_sub = clip_metrics_df.loc[clip_metrics_df['game_name'] == "Demon's Souls"]

In [ ]:
clip_metrics_df

In [ ]:
sub_group = clip_metrics_df.groupby('title').min()
game_sub = sub_group.loc[sub_group['game_name'] == "Demon's Souls"]

In [ ]:
clip_metrics_df

In [ ]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# Create server variable with Flask server object for use with gunicorn
server = app.server

app.layout = html.Div([
    html.Div([html.Label("Pick a Game to See Activity"),
    dcc.Dropdown(
                id='crossfilter-xaxis-column',
                options=[{'label': value, 'value': value} for key, value in games.items()],
                value="Demon's Souls"
            ),

    html.Div([dcc.Graph(id='graph')], style={'display': 'inline-block', 'padding': '0 20'})], 
    style = {'display': 'inline-block',  'margin': 'auto' }),
    
    


]
)

@app.callback(
    dash.dependencies.Output('graph', 'figure'),
    [dash.dependencies.Input("crossfilter-xaxis-column", "value")]
)
def update_figure(value):
    fig = make_subplots(rows=1, cols=4, shared_yaxes=True, vertical_spacing=0.02)
    sub_group = clip_metrics_df.groupby('title').min()
    game_sub = sub_group.loc[sub_group['game_name'] == value]
    sub_filter = clip_metrics_df.loc[clip_metrics_df['game_name'] == value]
    mobile_sub = sub_filter.loc[sub_filter['style'] == 'm']
    full_sub = sub_filter.loc[sub_filter['style'] == 'f']
    # creators = mobile_sub['creator'].unique()
    for creator, value in creators.items():
        fig.add_trace(go.Bar(x=game_sub.loc[game_sub.creator == creator, 'view_count'].values, y=game_sub.loc[game_sub.creator == creator].index, orientation='h', name=creator, legendgroup=creator, marker={'color': value}), row=1, col=1)
        fig.add_trace(go.Bar(x=mobile_sub.loc[mobile_sub.creator == creator, 'views'].values, y=mobile_sub.loc[mobile_sub.creator == creator, 'title'].values, orientation='h', name='Shorts', legendgroup = creator, showlegend=False, marker={'color': 'blue'} ), row=1, col=2)
        fig.add_trace(go.Bar(x=full_sub.loc[full_sub.creator == creator, 'views'].values, y=full_sub.loc[full_sub.creator == creator, 'title'].values, orientation='h', name='Full Screen',  legendgroup = creator, showlegend=False, marker={'color': 'purple'} ), row=1, col=2)
        fig.add_trace(go.Scatter(x=mobile_sub.loc[mobile_sub.creator == creator,'likes_ratio'].values, y=mobile_sub.loc[mobile_sub.creator == creator,'title'].values, mode='markers', name="Shorts", legendgroup = creator, showlegend=False, marker={'color': 'blue', 'size': mobile_sub.loc[mobile_sub.creator == creator,'likes'].values}), row=1, col=3)
        fig.add_trace(go.Scatter(x=full_sub.loc[full_sub.creator == creator,'likes_ratio'].values, y=full_sub.loc[full_sub.creator == creator,'title'].values, mode='markers', name="Full Screen", legendgroup = creator, showlegend=False, marker={'color': 'purple', 'size': full_sub.loc[full_sub.creator == creator,'likes'].values}), row=1, col=3)
        
        fig.add_trace(go.Bar(x=sub_filter.loc[sub_filter.creator == creator, 'subscribersGained'].values, y=sub_filter.loc[sub_filter.creator == creator,'title'].values, orientation='h', name=creator, legendgroup = creator, showlegend=False, marker={'color': 'green'} ), row=1, col=4)
        fig.add_trace(go.Bar(x=sub_filter.loc[sub_filter.creator == creator, 'subscribersLost'].values, y=sub_filter.loc[sub_filter.creator == creator,'title'].values, orientation='h', name=creator, legendgroup = creator, showlegend=False, marker={'color': 'red'} ), row=1, col=4)
    fig.update_layout( width=1000, height=1000, barmode='stack', xaxis3 = {'range': [0,1], 'tickformat': ".0%"}, xaxis4={'dtick': 1} )
    return fig

    # html.Div([html.Div([html.H1("Like Monitoring by Name")],  {'width': '49%', 'display': 'inline-block', 'margin': '10px', 'float': 'right'}),
        #     html.H1("Like Monitoring by Name"),
        #     dcc.Dropdown(
        #         id='crossfilter-xaxis-column',
        #         options=[{'label': value, 'value': value} for key, value in games.items()],
        #         value="Demon's Souls"
        #     )
        # ], style = {'width': '49%', 'display': 'inline-block', 'margin': '10px', 'float': 'right'})
        # # ])
        # # ])

app.run_server(mode='inline')

In [ ]:
def completeThumbnails(playlist, default_mode = 'default'):
    default = playlist['snippet']['thumbnails'][default_mode]
    checks = ['default', 'medium', 'high', 'standard']
    for chck in checks:
        if not bool(playlist['snippet']['thumbnails'].get(chck)):
            playlist['snippet']['thumbnails'][chck] = default
    return playlist

In [ ]:
playlists = service.playlists().list(part='id,snippet,status', channelId=os.environ['YT_CHANNEL_ID']).execute()
nextPage = playlists['nextPageToken']
while nextPage != None:
    seedPlaylist = service.playlists().list(part='id,snippet,status', channelId=os.environ['YT_CHANNEL_ID'], pageToken = nextPage).execute()
    playlists['items'] = playlists['items'] + seedPlaylist['items']
    if bool(seedPlaylist.get('nextPageToken')):
        nextPage = seedPlaylist['nextPageToken']
    else:
        nextPage = None

mapped_as_full_counts = {}
mapped_full_vids = {}
with Session(engine) as session:
    for title in session.query(Clip_Tracker.title).where(Clip_Tracker.published == PublishingStatus.f).order_by(Clip_Tracker.view_count.desc()).all():
        mapped_full_vids[title[0]] = []
        mapped_as_full_counts[title[0]] = 0
shorts_pattern = f"[sS]hort" # check for YT short videos
with Session(engine) as session: # initialize session to sql using context manager
    for playlist in playlists['items']: # loop through the differen playlists
        playlist = completeThumbnails(playlist)
        for key, thumbnail in playlist['snippet']['thumbnails'].items(): # first ensure that thumbnails are mapped in thumbnail table
            if not bool(session.query(Thumbnails.id).where(Thumbnails.asset == thumbnail['url']).first()):
                session.add(Thumbnails(asset=thumbnail['url'], width = thumbnail['width'], height=thumbnail['height']))
                session.commit()
            # map the thumbnail table ids to the playlist dictionary
            playlist['snippet']['thumbnails'][key]['id'] = session.query(Thumbnails.id).where(Thumbnails.asset == thumbnail['url']).first()[0]

        # if the playlist id is not in the table, add it
        if not bool(session.query(PlayList_yt).where(PlayList_yt.id == playlist['id']).first()):
            session.add(PlayList_yt(id=playlist['id'], title=playlist['snippet']['title'], 
            tb_default = playlist['snippet']['thumbnails']['default']['id'], 
            tb_medium = playlist['snippet']['thumbnails']['medium']['id'],
            tb_high = playlist['snippet']['thumbnails']['high']['id'], 
            tb_standard = playlist['snippet']['thumbnails']['standard']['id']
            ))
            session.commit()

        # else, update the table with the current values if it's already in the table - this is to capture if thumbnails or titles have changed
        else:
            session.execute(
                update(PlayList_yt).where(PlayList_yt.id == playlist['id']).values(dict(title=playlist['snippet']['title'], 
            tb_default = playlist['snippet']['thumbnails']['default']['id'], 
            tb_medium = playlist['snippet']['thumbnails']['medium']['id'],
            tb_high = playlist['snippet']['thumbnails']['high']['id'], 
            tb_standard = playlist['snippet']['thumbnails']['standard']['id']))
            )
            session.commit()
        if "Shorts" in playlist['snippet']['title']:
            style = VideoStyle.m
        else:
            style = VideoStyle.f
        # next find all the videos in each playlist
        playlist_Items = service.playlistItems().list(part="id,snippet", playlistId=playlist['id']).execute()
            
        nextPage = playlist_Items['nextPageToken']
        while nextPage != None:
            seed_list = service.playlistItems().list(part="id,snippet", playlistId=playlist['id'], pageToken = nextPage).execute()
            nextPage = seed_list['nextPageToken']
            playlist_Items['items'] = playlist_Items['items'] + seed_list['items']
        for video in playlist_Items['items']:
            try:
                video = completeThumbnails(video)
                title = video['snippet']['title']
                description = video['snippet']['description']
                    # check whether the title or description have the word "short" in it
                # combine the tags into a  single string due to datatype limitations in database - leave as array if your database can store arrays
                tags = " | ".join(service.videos().list(part='snippet,contentDetails', id=video['snippet']['resourceId']['videoId']).execute()['items'][0]['snippet']['tags'])
                

                # to capture titles with title conventon used in yt_videoupload.py
                title_components = title.split("|")
                if len(title_components) > 1:
                    titleName = title_components[-1].strip()
                
                else: # incase title does not follow current convention
                    titleName = title
                
                try:
                    mapped_as_full_counts[titleName] += 1
                    mapped_full_vids[titleName].append(style)
                except:
                    pass

            
                # find clip and original filename in clips database
                for clip_id, filename in session.query(Clip_Tracker.id, Clip_Tracker.video_name).where(Clip_Tracker.title == titleName).all():

                    if not bool(clip_id):
                        print(title)
                    else:
                        # loop through the thumbnail images to ensure they are in the thumbnail database
                        for key, thumbnail in video['snippet']['thumbnails'].items():
                            if not bool(session.query(Thumbnails.id).where(Thumbnails.asset == thumbnail['url']).first()):
                                session.add(Thumbnails(asset=thumbnail['url'], width = thumbnail['width'], height=thumbnail['height']))
                                session.commit()
                            video['snippet']['thumbnails'][key]['id'] = session.query(Thumbnails.id).where(Thumbnails.asset == thumbnail['url']).first()[0]
                        # VideoID is not in the Video_yt table - add it!    
                        if not bool(session.query(Video_yt).where(Video_yt.id == video['snippet']['resourceId']['videoId']).first()):
                            session.add( Video_yt(id = video['snippet']['resourceId']['videoId'], 
                            clip_id=clip_id, 
                            title = title,
                            filename = filename,
                            style = style,
                            description = description,
                            tb_default = video['snippet']['thumbnails']['default']['id'], 
                            tb_medium = video['snippet']['thumbnails']['medium']['id'],
                            tb_high = video['snippet']['thumbnails']['high']['id'], 
                            tb_standard = video['snippet']['thumbnails']['standard']['id'],
                            tags = tags
                            
                            )
                            )
                            session.commit()
                        # If it's already in there, update our record of current descriptiption / thumbnails.
                        else:
                            if style == VideoStyle.m:
                                print(video)
                            session.execute(
                                update(Video_yt).where(Video_yt.id == video['snippet']['resourceId']['videoId']).values(
                                    dict(
                                        clip_id=clip_id, 
                                        title = title,
                                        filename = filename,
                                        style = style,
                                        description = description,
                                        tb_default = video['snippet']['thumbnails']['default']['id'], 
                                        tb_medium = video['snippet']['thumbnails']['medium']['id'],
                                        tb_high = video['snippet']['thumbnails']['high']['id'], 
                                        tb_standard = video['snippet']['thumbnails']['standard']['id'],
                                        tags = tags
                                    )
                                )
                                )
                            session.commit()
                        if not bool(session.query(yt_vid_pl_mapper).where(yt_vid_pl_mapper.c.yt_video_id == video['snippet']['resourceId']['videoId']).where(yt_vid_pl_mapper.c.yt_playlist_id ==playlist['id']).first()):
                            session.execute(insert(yt_vid_pl_mapper).values({"yt_playlist_id": playlist['id'], 'yt_video_id': video['snippet']['resourceId']['videoId']}))
            except Exception as e:
                print("failed", e)

In [ ]:
playlist_Items['items'][0]['snippet']

In [ ]:
with Session(engine)as session:
    for clip_id, game_title, clip_creator, console, clip_title, video_name, clip_url, publishing_status in session.query(Clip_Tracker.id, Game_Meta.game_name, Clip_Tracker.creator_name, Game_Meta.platform, Clip_Tracker.title, Clip_Tracker.video_name, Clip_Tracker.url, Clip_Tracker.published).select_from(join(Game_Meta,Clip_Tracker)).where(and_(Clip_Tracker.published != PublishingStatus.f,  Clip_Tracker.published != None, Clip_Tracker.mobiles_videos_processed == True)).order_by(Clip_Tracker.view_count.desc()).all():
        print(clip_id)

In [ ]:
playlists = service.playlists().list(part='id,snippet,status', channelId=os.environ['YT_CHANNEL_ID']).execute()
nextPage = playlists['nextPageToken']
while nextPage != None:
    seedPlaylist = service.playlists().list(part='id,snippet,status', channelId=os.environ['YT_CHANNEL_ID'], pageToken = nextPage).execute()
    playlists['items'] = playlists['items'] + seedPlaylist['items']
    if bool(seedPlaylist.get('nextPageToken')):
        nextPage = seedPlaylist['nextPageToken']
    else:
        nextPage = None
all_items = {}
for playlist in playlists['items']: 
    playlist_Items = service.playlistItems().list(part="id,snippet", playlistId=playlist['id']).execute()
    all_items[playlist['id']] = {'playlist_items': None, "videos": []}
    all_items[playlist['id']]['playlist_items'] = playlist_Items
    if bool(playlist_Items.get('nextPageToken')):
        nextPage = playlist_Items['nextPageToken']
    else:
        nextPage = None
    while nextPage != None:
        seed_list = service.playlistItems().list(part="id,snippet", playlistId=playlist['id'], pageToken = nextPage).execute()
        if bool(seed_list.get('nextPageToken')):
            nextPage = seed_list['nextPageToken']
        else:
            nextPage = None
        playlist_Items['items'] = playlist_Items['items'] + seed_list['items']
        for video in playlist_Items['items']:
            all_items[playlist['id']]['videos'].append(service.videos().list(part='snippet,contentDetails', id=video['snippet']['resourceId']['videoId']).execute())

In [ ]:
for key, value in all_items.items():
    for value2 in value['videos']:
        if bool(value2['items']):
            if value2['items'][0]['kind'] != 'youtube#video':
                # if bool()
                print(value2)

In [ ]:
value2['items']

In [ ]:
with Session(engine) as session:
    print(session.query(Video_yt.title, Video_yt.))

In [ ]:
playlist_Items['items']

In [ ]:
with Session(engine) as session:
    for clip_id in session.query(Clip_Tracker.id).where(Clip_Tracker.published == PublishingStatus.f).all():
        rows = session.query(Video_yt.title, Video_yt.style).where(Video_yt.clip_id == clip_id[0]).all()
        if len(rows) < 2 and bool(rows):
            session.query(Clip_Tracker).where(Clip_Tracker.id == clip_id[0]).update(values = {'published': invers_mapper[rows[0][1]]})
            session.commit()

            print(rows[0])

In [ ]:
rows[0]